In [560]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import statsmodels.api as sm2
import statistics as stat
import time
from numpy.random import normal
import seaborn as sns
import pylab
from scipy import optimize as opt
from scipy import stats as st
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.tsaplots import plot_pacf,plot_acf
from matplotlib.pyplot import figure
from sklearn.decomposition import PCA
from bisect import bisect_left
from tabulate import tabulate
import timeit
import scipy 
import math
import pypfopt

# Problem 1

In [543]:
price=100
n=10000
std=0.1
r=st.norm(0,std).rvs(n)
price1=price+r
price2=price*(1+r)
price3=price*np.exp(r)

print("Classical Brownian Motion: " )
print("mean= ", round(np.mean(price1),2), " std= ",round(np.std(price1),2))
print()
print("Arithmetic: " )
print("mean= ", round(np.mean(price2),2), " std= ",round(np.std(price2),2))
print()
print("Log Return " )
print("mean= ", round(np.mean(price3),2), " std= ",round(np.std(price3),2))
print("mean(ln(price3))= ", round(np.mean(np.log(price3)),2), " std(ln(price3)= ",round(np.std(np.log(price3)),2))

Classical Brownian Motion: 
mean=  100.0  std=  0.1

Arithmetic: 
mean=  100.1  std=  10.15

Log Return 
mean=  100.62  std=  10.23
mean(ln(price3))=  4.61  std(ln(price3)=  0.1


# Problem 2

In [544]:
df=pd.read_csv('https://raw.githubusercontent.com/dompazz/FinTech590-RiskManagement/main/Week04/DailyPrices.csv')
df=df.iloc[:,1:]
intc=df["INTC"]
intc=intc.to_frame()
intc

,INTC
0,54.987606
1,55.613255
2,49.118420
3,49.068768
4,47.946571
...,...
56,55.209999
57,55.910000
58,55.740002
59,54.939999


In [545]:
#Function that calculate the exponential weight
def populateWeights(x,w,cw, λ):
    n = len(x)
    tw = 0.0
    for i in range(1,n+1):
        x[i-1] = i
        w[i-1] = (1-λ)*λ**i
        tw += w[i-1]
        cw[i-1] = tw
        
    for i in range(1,n+1):
        w[i-1] = w[i-1]/tw
        cw[i-1] = cw[i-1]/tw


#Function that generate the exponentially weighted covariance matrix
def out_put_EWCM(w,n,df):
    cov_matrix = np.zeros([n,n])
    for i in range (df.shape[0]):
        for j in range (n):
            df.iloc[i,j] = df.iloc[i,j] - np.mean(df.iloc[:,j])
    
            
    for i in range (n):
        for j in range (n):
            temp = w * df.iloc[:,i]
            cov_matrix[i,j] = np.dot(temp, df.iloc[:, j])
            
    return cov_matrix

n=61
x=[0.0] * n
w=[0.0] * n
cw=[0.0] * n

In [546]:
#function that calculates return
def return_calculate(data,method = 'DISCRETE'):
    vars = data
    p = np.matrix(vars)
    n = p.shape[0]
    m = p.shape[1]
    p2 = np.zeros((n-1,m))
    
    for i in range(n-1):
        for j in range(m):
            p2[i][j] = p[i+1][j] / p[i][j]
    
    if method.upper() == 'DISCRETE':
        p2 -= 1
    elif method.upper() == 'Log':
        p2 = np.log(p2)
    else:
        print('method should be Log or DISCRETE')
    return p2

In [547]:
#Calcukate return for INTC
ret_intc=return_calculate(intc)

In [548]:
#Remove mean
adj_ret_intc=ret_intc-np.mean(ret_intc)
#adj_ret_intc

In [549]:
#Calculate VaR using normal distribution

intc_std=np.std(adj_ret_intc)

# 1. Normal
ret_norm_sim=st.norm(0,intc_std).rvs(10000)
VaR_Norm=-np.quantile(ret_norm_sim,q=0.05)
print("1. VaR using Normal Distribution is :",VaR_Norm*100,"%")

# 2. Exp
ew_var=pd.DataFrame(adj_ret_intc).ewm(alpha=0.06).var()
ew_std=np.sqrt(ew_var).iloc[-1,:]
VaR_Exp_sim=st.norm(0,ew_std).rvs(10000)
VaR_Exp=-np.quantile(VaR_Exp_sim,q=0.05)
print("2. VaR using Exp weighted Normal Distribution is :",VaR_Exp*100,"%")


# 3. MLE
def mle_t(param_vec):
    std,n=param_vec[0],param_vec[1]
    negll=-np.sum(st.t.logpdf(adj_ret_intc,df=n,scale=std))
    return negll
#optimization(minimization)
model_t = opt.minimize(mle_t, np.array([1, 1]))
mle_std=model_t.x[0]
mle_n=model_t.x[1]
mle_sim=st.t(df=mle_n,scale=mle_std).rvs(10000)
VaR_Mle=-np.quantile(mle_sim,q=0.05)
print("3. VaR using MLE fitted T distribution is :",VaR_Mle*100, "%")



# 4. Historical
VaR_His=-np.quantile(adj_ret_intc,q=0.05)
print("4. VaR using Historical Simulation is :",VaR_His*100,"%")


1. VaR using Normal Distribution is : 3.374615733932757 %
2. VaR using Exp weighted Normal Distribution is : 2.5934884568969676 %
3. VaR using MLE fitted T distribution is : 2.8448669572534375 %
4. VaR using Historical Simulation is : 2.072516857535336 %


In [550]:
Yahoo=pd.read_csv("INTC.csv")

In [551]:
Yahoo=Yahoo.iloc[:,1:]
Yahoo_ret=return_calculate(Yahoo["Adj Close"].to_frame())
Yahoo_ret-=np.mean(Yahoo_ret)

In [552]:

fig,axes=plt.subplots(1,2,figsize=(20,5),dpi=1000)
sns.histplot(Yahoo_ret, stat="density",ax=axes[0],label='Yahoo_Future',color='grey',alpha=0.5)
sns.distplot(ret_norm_sim,ax=axes[0],hist=False,label='Normal',color='orange')
sns.distplot(VaR_Exp_sim,ax=axes[0],hist=False,label='EXP',color='blue')
sns.distplot(mle_sim,ax=axes[0],hist=False,label='MLE',color='red')

sns.histplot(adj_ret_intc, stat="density",ax=axes[1],label='Historical',color='grey',alpha=0.5)
sns.distplot(ret_norm_sim,ax=axes[1],hist=False,label='Normal',color='orange')
sns.distplot(VaR_Exp_sim,ax=axes[1],hist=False,label='EXP',color='blue')
sns.distplot(mle_sim,ax=axes[1],hist=False,label='MLE',color='red')

axes[0].legend()
axes[0].set_xlabel("Adj_Return")
axes[0].set_title("Comparison based on Future Prices")
axes[1].legend()
axes[1].set_xlabel("Adj_Return")
axes[1].set_title("Comparison based on Historical Prices")
axes[0].set_xticks(np.arange(-0.2, 0.2, step=0.05))



/Users/fredpeng/opt/anaconda3/lib/python3.8/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/Users/fredpeng/opt/anaconda3/lib/python3.8/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/Users/fredpeng/opt/anaconda3/lib/python3.8/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-leve

# Problem 3

In [553]:
portfolio=pd.read_csv("https://raw.githubusercontent.com/dompazz/FinTech590-RiskManagement/main/Week04/Project/portfolio.csv")
portfolio                                        

,Portfolio,Stock,Holding
0,A,AAPL,58
1,A,TSLA,10
2,A,FB,31
3,A,JPM,64
4,A,HD,27
...,...,...,...
95,C,F,397
96,C,LRCX,14
97,C,MO,197
98,C,LMT,27


In [554]:
P_A=portfolio[portfolio['Portfolio']=='A']
P_B=portfolio[portfolio['Portfolio']=='B']
P_C=portfolio[portfolio['Portfolio']=='C']

In [559]:
fig,axes=plt.subplots(1,3,figsize=(20,5),dpi=1000)
def his(a,ret,cp,port):
    vHoldings=port['Holding'].tolist()
    p_val=((1+ret)*cp)@vHoldings
    p_val = p_val.sort_values()
    p_val=p_val.reset_index(drop=True)
    pv=(cp*vHoldings).sum()
    return pv-p_val[int(a*len(ret))]


portfolio_list=[P_A,P_B,P_C]
his_list=[]
for i in range(len(portfolio_list)):
    data=portfolio_list[i]
    Stock=portfolio_list[i]['Stock'].tolist()
    prices=df[Stock]
    ret=prices.copy()
    sns.distplot(ret[portfolio_list[i].Stock],ax=axes[i],hist=False,label='EXP',color='blue')
    for col in prices.columns[0:]:
        ret[col]=prices[col]/prices[col].shift()-1
    cp=prices.iloc[len(prices)-1,:]
    #print(ret)
    VaR=his(0.05,ret,cp,portfolio_list[i])
    his_list.append(VaR)

/Users/fredpeng/opt/anaconda3/lib/python3.8/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/Users/fredpeng/opt/anaconda3/lib/python3.8/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/Users/fredpeng/opt/anaconda3/lib/python3.8/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-leve

In [556]:
Stock=portfolio["Stock"].tolist() 
prices=df[Stock]
ret=prices.copy()
for col in prices.columns[0:]:
        ret[col]=prices[col]/prices[col].shift()-1
cp=prices.iloc[len(prices)-1,:]
    #print(ret)
Total_VaR=his(0.05,ret,cp,portfolio)

In [557]:
his_list.append(Total_VaR)


  
mydata = [{ his_list[0],"A"}, 
          { his_list[1],"B"}, 
          { his_list[2],"C"}, 
          { his_list[3],"Total"}]
  

head = ["VaR","Portfolio"]
  

print(tabulate(mydata, headers=head, tablefmt="grid"))

+----------+-------------+
|      VaR | Portfolio   |
+==========+=============+
|  5298.49 | A           |
+----------+-------------+
|  5576.13 | B           |
+----------+-------------+
|  3307.76 | C           |
+----------+-------------+
| 12460.9  | Total       |
+----------+-------------+
